In [1]:
import kwant
import kwant.continuum
import numpy as np
import sympy
import matplotlib.pyplot as plt
from sympy import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Description of Code

$$n(E) = \int \rho(E)F(E) dE$$
where $n(E)$ is the total number of states filled, $\rho(E)$ is the density of states, $F(E)$ is the Fermi Function

We take the system *syst* to be made from the template by discretizing the continuous Hamiltonian. We take the system to be of the dimensions $25 \times 25$.

We then first plot for a range of temperatures (0 to 50) at Magnetic Field B = 0, $\Delta i = 0$, and $\Delta h = 0$ the number of states filled.

After that, we plot for a range of magnetic field values, namely (-5 to 5), for a temperature value of 0.2K.

The Code works by using the Kernel-Polynomial Method to obtain the spectral density of the system which was previously made from the Dirac Hamiltonian. Then, we take the fermi function to be $$F(E) = \frac{1}{e^{(E - E_f) / kT} + 1}$$ and we utilize the spectrum.integrate function to sum up all possible density of states to finally obtain the total number of states occupied.

In [2]:
'''
Function: Solving for n vs. B where n is the carrier density 
n(E) = integral of D(E)F(E) where D is the density of states and F is the fermi function
'''
def plotnvst(inv = 0, deltaH = 0):
    def make_syst(a=1):
        hamiltonian = '''
                        h * v_f * Matrix(BlockMatrix([ [ sigma_x * k_y - k_x * sigma_y , zeros(2)], [zeros(2), -1 * (sigma_x * k_y - k_x * sigma_y)] ])) 
                        + Matrix([[inv, 0,0,0] , [0, invconj, 0 , 0 ] , [ 0, 0, -inv, 0] , [0 , 0 , 0, negInvConj]] )
                        + Matrix([[0, 0,0,deltaH] , [0,0  ,conjH , 0 ] , [ 0, deltaH, 0, 0] , [conjH, 0 ,0, 0]] )
                        + Matrix(BlockMatrix( [ [g * mu * B * sigma_z, zeros(2)], [zeros(2), g * mu * B * sigma_z]]))
                        '''
        template = kwant.continuum.discretize(hamiltonian, grid=a)
        L, W = 25,25

        def shape(site):
            (x, y) = site.pos
            return (0 <= y < W and 0 <= x < L)

        syst = kwant.Builder()
        syst.fill(template, shape, (0, 0))

        syst.eradicate_dangling()

        return syst

    numStates = []

    inv = inv
    invconj = conjugate(inv)
    negInvConj = conjugate(-inv)

    h = 1
    deltaH = deltaH
    conjH = conjugate(deltaH)
    v_f = 1

    g = 1
    mu = 1
    b_values = np.linspace(-5,5, 25)
    temps = np.linspace(0,50,50)
    for t in temps:
        syst = make_syst().finalized()
        params = dict(v_f = v_f, h =h, inv = inv, invconj = invconj, negInvConj = negInvConj,conjH = conjH, deltaH = deltaH, g =g , mu = mu, B = 0)
        spectrum = kwant.kpm.SpectralDensity(syst,params)
        #energies, densities = spectrum()
        fermi = lambda E: 1 / (np.exp((E - 0.1) / t) + 1)
        numStates.append(spectrum.integrate(fermi))

    plt.plot(temps, numStates)
    plt.xlabel("Temperature (K)")
    plt.ylabel("# of Filled States")
    plt.title("Total Number of Filled States vs. Temperature")

In [3]:
interactive_plot = interactive(plotnvst, inv = (0,5,0.05), deltaH = (0,5,0.05))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='inv', max=5.0, step=0.05), FloatSlider(value=0.0, de…

In [8]:
'''
Function: Solving for n vs. B where n is the carrier density 
n(E) = integral of D(E)F(E) where D is the density of states and F is the fermi function
'''

def plotnvsb(inv = 0, deltaH = 0):
    def make_syst(a=1):
        hamiltonian = '''
                        h * v_f * Matrix(BlockMatrix([ [ sigma_x * k_y - k_x * sigma_y , zeros(2)], [zeros(2), -1 * (sigma_x * k_y - k_x * sigma_y)] ])) 
                        + Matrix([[inv, 0,0,0] , [0, invconj, 0 , 0 ] , [ 0, 0, -inv, 0] , [0 , 0 , 0, negInvConj]] )
                        + Matrix([[0, 0,0,deltaH] , [0,0  ,conjH , 0 ] , [ 0, deltaH, 0, 0] , [conjH, 0 ,0, 0]] )
                        + Matrix(BlockMatrix( [ [g * mu * B * sigma_z, zeros(2)], [zeros(2), g * mu * B * sigma_z]]))
                        '''
        template = kwant.continuum.discretize(hamiltonian, grid=a)
        L, W = 25,25

        def shape(site):
            (x, y) = site.pos
            return (0 <= y < W and 0 <= x < L)

        syst = kwant.Builder()
        syst.fill(template, shape, (0, 0))

        syst.eradicate_dangling()

        return syst

    numStates = []

    inv = inv
    invconj = conjugate(inv)
    negInvConj = conjugate(-inv)

    h = 1
    deltaH = deltaH
    conjH = conjugate(deltaH)
    v_f = 1

    g = 1
    mu = 1
    b_values = np.linspace(-5,5, 10)
    temps = np.linspace(0,50,50)
    for B in b_values:
        syst = make_syst().finalized()
        params = dict(v_f = v_f, h =h, inv = inv, invconj = invconj, negInvConj = negInvConj,conjH = conjH, deltaH = deltaH, g =g , mu = mu, B = B)
        spectrum = kwant.kpm.SpectralDensity(syst,params)
        energies, densities = spectrum()
        fermi = lambda E: 1 / (np.exp((E - 0.003) / 0.2) + 1)
        numStates.append(spectrum.integrate(fermi)) 
    params = dict(v_f = v_f, h =h, inv = inv, invconj = invconj, negInvConj = negInvConj,conjH = conjH, deltaH = deltaH, g =g , mu = mu, B = 0)
    fsyst_staggered = make_syst().finalized()
    # find 'A' and 'B' sites in the unit cell at the center of the disk
    #center_tag = np.array([0, 0])
    #where = lambda s: s.tag == center_tag
    # make local vectors
    #vector_factory = kwant.kpm.LocalVectors(fsyst_staggered, where)
    #local_dos = kwant.kpm.SpectralDensity(fsyst_staggered, num_vectors=None,
                                      #vector_factory=vector_factory,
                                      #params = params,
                                      #mean=False,
                                      #rng=0)
   # energies, densities = local_dos()
    #plt.plot(energies, densities[:,0])
    #plt.plot(energies, densities[:,1])
    plt.plot(b_values, numStates)
    plt.xlabel("Energy")
    plt.ylabel("Density of States")
    plt.title("Density of States vs. Magnetic Field at 0.2K")
    plt.show()

In [9]:
interactive_plot = interactive(plotnvsb, inv = (0,5,0.05), deltaH = (0,5,0.05))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='inv', max=5.0, step=0.05), FloatSlider(value=0.0, de…